In [3]:
import pandas as pd
import random
from datetime import datetime, timedelta

In [4]:
# Define logical mapping of incidents, controls, and control failures
incident_control_mapping = {
    "Unauthorized Access": ("Access Control", "Access Control Failure"),
    "Data Leakage": ("Data Loss Prevention", "Insufficient DLP Configuration"),
    "Malware Infection": ("Antivirus Software", "Antivirus Not Updated"),
    "Phishing Attempt": ("User Awareness Training", "Inadequate User Training"),
    "Denial of Service": ("Network Firewall", "Firewall Misconfiguration"),
    "Privilege Escalation": ("User Privilege Management", "Weak Privilege Controls"),
    "Data Breach": ("Encryption", "Lack of Data Encryption"),
    "SQL Injection": ("Input Validation", "Inadequate Input Validation"),
    "Password Attack": ("Authentication Controls", "Weak Authentication Policies"),
    "Insider Threat": ("Monitoring and Logging", "Lack of Monitoring")
}

# Create a larger dataset with 100 entries
num_entries = 100

# Generate random data
data = {
    "Incident": [random.choice(list(incident_control_mapping.keys())) for _ in range(num_entries)],
}

# Assign controls and control failures based on the selected incident
data["Control"] = [incident_control_mapping[incident][0] for incident in data["Incident"]]
data["Control Failure"] = [incident_control_mapping[incident][1] for incident in data["Incident"]]

# Add remaining fields
cia_combinations = ["MMM", "LLL", "HHH", "MLH", "LMH", "HLM", "MHL", "LHM", "HMM", "LLH"]
lob_values = ["Upstream", "Downstream", "Global Functions", "Enterprise Technology"]

data["CIA Rating"] = [random.choice(cia_combinations) for _ in range(num_entries)]
data["L&R Flags (T/F)"] = [random.choice([True, False]) for _ in range(num_entries)]
data["Attestation"] = [random.choice(["Pending", "Non-Compliant", "Compliant"]) for _ in range(num_entries)]
data["DE Result"] = [random.choice(["Pass", "Fail", "N/A"]) for _ in range(num_entries)]
data["OE Result"] = [random.choice(["Pass", "Fail", "N/A"]) for _ in range(num_entries)]
data["Line of Business"] = [random.choice(lob_values) for _ in range(num_entries)]
data["Date"] = [(datetime.now() - timedelta(days=random.randint(1, 365))).date() for _ in range(num_entries)]
data["Time to Detect (hrs)"] = [random.randint(1, 72) for _ in range(num_entries)]
data["Time to Respond (hrs)"] = [random.randint(1, 48) for _ in range(num_entries)]

# Create a DataFrame
df_large = pd.DataFrame(data)

# Add the new column for Information Missing/Not Updated
df_large["Information Missing/Not Updated"] = df_large.apply(
    lambda row: "Missing Values"
    if row["Attestation"] in ["Pending", "Non-Compliant"] or row["DE Result"] == "N/A" or row["OE Result"] == "N/A"
    else "",
    axis=1
)

# Add the new column for Business Critical
df_large["Business Critical"] = df_large["CIA Rating"].apply(
    lambda x: "Yes" if "H" in x else "No"
)

# Function to determine Inherent Risk based on CIA values
def calculate_inherent_risk(cia_rating):
    h_count = cia_rating.count('H')
    if h_count > 2:
        return 'Very High'
    elif h_count == 2:
        return 'High'
    elif 'M' in cia_rating:
        return 'Medium'
    else:
        return 'Low'

# Create Inherent Risk column
df_large["Inherent Risk"] = df_large["CIA Rating"].apply(calculate_inherent_risk)


# Display the first few rows of the updated dataset
df_large


,Incident,Control,Control Failure,CIA Rating,L&R Flags (T/F),Attestation,DE Result,OE Result,Line of Business,Date,Time to Detect (hrs),Time to Respond (hrs),Information Missing/Not Updated,Business Critical,Inherent Risk
0,Password Attack,Authentication Controls,Weak Authentication Policies,MHL,False,Pending,Pass,Pass,Upstream,2024-02-03,46,26,Missing Values,Yes,Medium
1,Malware Infection,Antivirus Software,Antivirus Not Updated,HHH,True,Compliant,N/A,Pass,Upstream,2023-11-20,11,26,Missing Values,Yes,Very High
2,Malware Infection,Antivirus Software,Antivirus Not Updated,MLH,False,Non-Compliant,N/A,Fail,Global Functions,2024-03-20,59,39,Missing Values,Yes,Medium
3,Data Breach,Encryption,Lack of Data Encryption,MHL,True,Compliant,N/A,Pass,Upstream,2023-11-10,32,22,Missing Values,Yes,Medium
4,Insider Threat,Monitoring and Logging,Lack of Monitoring,LMH,True,Compliant,Pass,N/A,Global Functions,2024-09-04,47,42,Missing Values,Yes,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Denial of Service,Network Firewall,Firewall Misconfiguration,HLM,True,Compliant,N/A,N/A,Enterprise Technology,2024-06-13,25,9,Missing Values,Yes,Medium
96,Data Leakage,Data Loss Prevention,Insufficient DLP Configuration,HLM,True,Compliant,N/A,N/A,Global Functions,2024-07-02,4,9,Missing Values,Yes,Medium
97,Privilege Escalation,User Privilege Management,Weak Privilege Controls,LMH,True,Pending,Pass,Fail,Global Functions,2024-03-21,28,38,Missing Values,Yes,Medium
98,Unauthorized Access,Access Control,Access Control Failure,HHH,True,Non-Compliant,Pass,Pass,Upstream,2024-03-10,5,11,Missing Values,Yes,Very High


In [5]:
df_large.to_csv('control_effectiveness.csv')